In [14]:
#!pip install bokeh on a standard python install
from bokeh.layouts import row,column
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook,push_notebook
from microtsetlin import *
import time
output_notebook()

Loading BokehJS ...

## Microtsetlin

This notebook contains WIP visualisation of tsetlin states using hinton diagrams - it's not completely self-explanatory
as there's no iteration variables to explain where we are in the trianing cycle

TODO:
1. output iteration
2. output input and desired out put

In [16]:
def hinton(f, matrix, thres,max_weight=105):
    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))
    out = [(y,x,w / max_weight, "black" if w > thres else "white") for (x, y), w in np.ndenumerate(matrix)]
    x,y,s,c = zip(*out)
    h = f.rect(x, y, fill_color=list(c),line_color="black", width=s, height=s)
    return f,h 

def uhinton(target, h,matrix, thres, max_weight=105):
    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))
    out = [(y,x,w/float(max_weight), "black" if w > thres else "white") for (x, y), w in np.ndenumerate(matrix)]
    x,y,s,c = zip(*out)
    h.data_source.data['x']= x
    h.data_source.data['y']= y
    h.data_source.data['width']= s
    h.data_source.data['height']= s
    h.data_source.data['fill_color']= list(c)
    push_notebook(handle=target)

In [20]:
a = ['E','I']
f = figure(title="T inc", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500,x_axis_location="above")
f,h = hinton(f,tsetlin[:,:,0],MIDSTATE)
#f.xaxis.visible = False 
f.yaxis.visible = False
f.xaxis.ticker = [0, 1]
f.xaxis.major_label_overrides = {0: 'X1', 1: 'X2'}
f2 = figure(title="T inc neg", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500,x_axis_location="above")
f2,h2 = hinton(f2,tsetlin[:,:,1],MIDSTATE)
#f2.xaxis.visible = False
f2.xaxis.ticker = [0, 1]
f2.xaxis.major_label_overrides = {0: '¬X1', 1: '¬X2'}
f2.yaxis.visible = False
f3 = figure(title="Out", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500,x_axis_location="above")
f3,h3 = hinton(f3,np.array(num_clauses*[0]).reshape(1, num_clauses),0.5)
f3.xaxis.ticker = [0]
f3.xaxis.major_label_overrides = {0: 'Clause Out'}
f3.yaxis.visible = False
f4 = figure(title="In", toolbar_location=None,x_range=(-1.5,2.5),plot_width=200, plot_height=500,x_axis_location="above")
f4,h4 = hinton(f4,np.array(20*[0]).reshape(10, 2),0.5)
f4.xaxis.ticker = [0,1]
f4.xaxis.major_label_overrides = {0: 'X1', 1: 'X2'}
f4.yaxis.visible = False

# Tsetlin Machines: The Gory Details!

The general online learning algorithm:

![Tsetlin vs MLP](MLPTSET.png)

In [22]:
target = show(row(f4,f,f2,f3), notebook_handle=True)
tsetlin       = np.random.choice([MIDSTATE, MIDSTATE+1], size=(num_clauses,num_features, len([LIT,NOT_LIT])))  
iterations = 100
def train_vis():
    for i in range(iterations):
        e = 0
        out = []
        for x_hat,y_hat in zip(X,Y):
            clause_output = calc_all_clause_outputs(x_hat, tsetlin)
            tot_c_outputs = np.sum([c_out*sign if live else 0 for (c_out,live),sign in zip(clause_output,clause_sign)])
            out += [tot_c_outputs]
            y_est = 1 if tot_c_outputs >= 0 else 0 #nte: total 0 = output 1
            e += 1 if y_est != y_hat else 0
            if np.random.random() < 0.1:
                y_hat = 1 - y_hat
            update(tot_c_outputs,y_hat,polarity,clause_output,x_hat,tsetlin,T=4)
            uhinton(target,h4,50*np.repeat(np.array(x_hat).reshape(1,2),10,axis=0), 0.5)
            uhinton(target,h,tsetlin[:,:,0],  MIDSTATE)
            uhinton(target,h2,tsetlin[:,:,1], MIDSTATE)
            actual_c,_ = zip(*clause_output)
            uhinton(target,h3,50*np.array(actual_c).reshape(num_clauses,1), 0.5)
            time.sleep(0.1)

            #print("it", i, e,out)
    #print(np.transpose(tsetlin))    
train_vis()    

## Why are we interested in hardware Tsetlin?

- Smallest available "learning" footprint
    - Learning rules are lookup
    - State is represented as  counters
    - Inference is propositional logic
    - Flexible, can be used for many styles of learning including "deep" setups

- Explainable
    - Inference is propositional logic so can be read directly
    - Ensembles can be treated as probabilities

- Controllable
    - Hardware can be built to be power proportional
        - Algorithm iterations/ensemble size are big driver
    - Algorithm is simple so has discrete pause points

- Maps well "from hardware"
    - No FP
    - ANDs/ADD are the major operations
    - Async properties are nice

![quick comparison](MLPTSET2.png)


| Method        | Trees/Forest          | KNN           | Descriminant  | DL            | Tsetlin |
| ------------- |:---------------------:|:-------------:|:-------------:|:-------------:|:-------:|
| Representation| NODES/RULES           | DATA          | DATA          | WEIGHTS       | AUTOMATA |
| Hardware      | GINI/entropy          | Distance calcs|               | Graphs/Macs   | Boolean/SUM |
| Algorithm     | sequential entropy    | tree lookup   | projected     | bp grad       | game optimiser | 
| Flexibility   | Class/Reg             | Class/Reg     | Class         | Class/Reg/Gen/Re/Rec/Conv | AS DL |
| Base Type     | F                     | F             | F             | F             | I/B     |
| Explainabale  | Y                     | ish           | N             | N++           | Y        |
| Power         | 4                     | 2             | 3             | 5             | 5       |
| Issues...     | GINI                  | keep all data | inversion     | complexity/params | Discrete |



- Downsides
    - Some stochasticity...
    - Some discreteness

What we have:
- Highly optimised algorithm
- Several hardware design idea ranging from digital to ``temporal'' 
- Strong links with the originating researchers

Proposed next steps:

- Research
    - Removed/reduced stochasticity
    - Compressed ensembles
    - Localising memory fetches (in memory) 

- Development
    - Test design ideas on an initial problem
        - we are flexible to what this might be!